<a href="https://colab.research.google.com/github/manavmittal05/Large-Language-Models/blob/main/A1/Q1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain
!pip install git+https://github.com/huggingface/transformers.git
!pip install git+https://github.com/huggingface/accelerate.git
!pip install sentence-transformers==2.2.2
!pip install bitsandbytes
!pip install datasets
!pip install -qU langchain-huggingface
!pip install -qU "langchain-chroma>=0.1.2"
!pip install langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.4/396.4 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.0/290.0 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.8 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-ak1_xts7
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-b

In [4]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
import os
import random
import string
import warnings
from datasets import load_dataset
from langchain import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

warnings.filterwarnings('ignore')

### Vector Database creation

In [5]:
embedding_model_id = 'sentence-transformers/all-MiniLM-L6-v2'

embedding_model = HuggingFaceEmbeddings(
    model_name=embedding_model_id,
    model_kwargs={'device':'cuda'},
    encode_kwargs={'device': 'cuda', 'batch_size': 16}
)

In [6]:
vector_store = Chroma(
    collection_name="example_collection",
    embedding_function=embedding_model,
    persist_directory="./chroma_langchain_db",
)

In [83]:
from uuid import uuid4

from langchain_core.documents import Document

document_1 = Document(
    page_content="Sir Keir Starmer became Prime Minister on 5 July 2024. The Prime Minister is the leader of His Majesty’s Government and is ultimately responsible for the policy and decisions of the government.",
    metadata={"source": "website"},
    id=1,
)

document_2 = Document(
    page_content="Question: One side of the bottom layer of a triangular pyramid has 11 balls. How many are there in the whole pyramid? Note that the pyramid is equilateral and solid.\nAnswer: As there are 11 balls along one side, it means that there are 11 layers of balls. The top most layer has 1 ball. The second layer has 3 (1+2) balls. The third layer has 6 (1+2+3) balls. The fourth layer has 10 (1+2+3+4) balls. The fifth layer has 15 (1+2+3+4+5) balls. Similarly, there are 21, 28, 36, 45, 55 and 66 balls in the remaining layers. Hence, the total number of balls is = 1 + 3 + 6 + 10 + 15 + 21 + 28 + 36 + 45 + 55 + 66 = 286 balls",
    metadata={"source": "website"},
    id=2,
)

document_3 = Document(
    page_content="Question: When a number is divided by 5, the remainder is 3. What is the remainder when twice the number is divided by 5? Answer: 1",
    metadata={"source": "website"},
    id=3,
)

fact_check1 = Document(
    page_content="NITI Aayog recently released a “Pathways and Strategies for Accelerating Growth in Edible Oils towards Goal of Atma Nirbharta” report. The report reviews the current state of the sector, its potential, and strategies to address challenges. Nine major oilseed crops cover 14.3% of the cropped area, contributing 12-13% to dietary energy and 8% to agricultural exports. Soybean leads in production, followed by rapeseed-mustard and groundnut. Rajasthan and Madhya Pradesh are the top producers, with Gujarat and Maharashtra also significant contributors. Rising consumption has led to increased imports, with palm oil making up 59% of imports. Oilseed production and yield have been growing, with projections to reach 43 MT by 2030 and 55 MT by 2047.",
    metadata={"source": "news"},
    id=4,
)

fact_check2 = Document(
    page_content="Indian wrestler Vinesh Phogat, 29, announced her retirement after being disqualified from the Olympics for being 100 grams overweight before her gold medal bout in the 50 kg category. She appealed to the Court of Arbitration for Sports (CAS) to be awarded a joint silver medal, citing severe dehydration from extreme weight-cutting measures. Phogat, a three-time Olympian in different weight classes, became the first international wrestler to defeat reigning champion Yui Susaki and the first Indian female wrestler to reach an Olympic final in 2024. She is a multiple Commonwealth Games gold medalist and has won two World Wrestling Championship bronze medals.",
    metadata={"source": "news"},
    id=5,
)

fact_check3 = Document(
    page_content="Archaeologists have discovered a terracotta pipeline during the 10th phase of excavation at Keeladi, a Sangam Era site in Tamil Nadu. This find highlights advanced water management practices from over 2,600 years ago. Keeladi, located near Madurai, has revealed over 20,000 artifacts, suggesting the Sangam Era began around 600 BCE, earlier than previously thought. Additional discoveries include rice husks from 3,200 years ago and evidence of early iron technology.",
    metadata={"source": "news"},
    id=6,
)

fact_check4 = Document(
    page_content="The 7th Indian Coast Guard (ICG) Subordinate Officers’ Conclave, held on August 1-2, 2024, in New Delhi, was inaugurated by Director General Rakesh Pal. Themed ‘Towards an Inclusive Approach,’ the event included sessions on holistic living, stress management, and leadership. It provided a platform for officers to share ideas with decision-makers, focusing on inclusive growth, technology integration, and the principles of Aatmanirbhar Bharat and good governance.",
    metadata={"source": "news"},
    id=7,
)

fact_check5 = Document(
    page_content="It is given that the two digits of the required number are prime numbers i.e. 2, 3, 5 or 7. Note that 1 is neither prime nor composite. Also, the third digit is the multiplication of the first two digits. Thus, hundreds digit and units digit must be either 2 or 3 i.e. 2_2, 2_3, 3_2 or 3_3 which means that there are four possible numbers - 242, 263, 362 and 393. Now, it is also given that - the difference between its reverse and it is 99. So 263 and 362 satisfy this condition. Hence, the sum of the three digits is 11 in each case.",
    metadata={"source": "website"},
    id=8,
)

fact_check6 = Document(
    page_content="The Union Ministry of Power launched three new online platforms: PROMPT (Portal for Online Monitoring of Projects—Thermal), DRIPS (Disaster Resource Inventory for Power Sector), and JAL VIDYUT DPR, to improve the power sector’s efficiency. It has developed by the Central Electricity Authority under the Ministry of Power with NTPC’s help for PROMPT, these platforms aim to improve efficiency and transparency in the power sector. PROMPT tracks and analyzes thermal power projects in real-time to address construction delays. JAL VIDYUT DPR monitors the status of hydroelectric and pumped storage projects to ensure better management. DRIPS helps identify and manage power disruptions from natural disasters by linking nodal officers and agencies.",
    metadata={"source": "news"},
    id=9,
)

fact_check7 = Document(
    page_content="Air India is establishing South Asia’s largest flight training school in Maharashtra’s Amravati district with an investment of over ₹200 crore. Opening in the next financial year’s first quarter, it will address the airline’s need for 500-700 pilots annually, following a large aircraft order. In partnership with Maharashtra Airport Development Company (MADC), the 10-acre facility will train 180 commercial pilots yearly using 31 single-engine Piper aircraft and three twin-engine Diamond aircraft.",
    metadata={"source": "news"},
    id=10,
)

fact_check8 = Document(
    page_content="Russia’s Defence Ministry reported that Russian warplanes intercepted two U.S. strategic bombers over the Barents Sea, preventing a border violation. The Barents Sea, a marginal sea of the Arctic Ocean, covers 1.4 million sq. km and borders Norway and Russia. Named after Dutch explorer Willem Barents, it was known to Vikings as the Murmean Sea. It is bounded by several archipelagos and seas, including the Svalbard and Novaya Zemlya archipelagos.",
    metadata={"source": "news"},
    id=11,
)

fact_check9 = Document(
    page_content="The Daksha project plans to construct two high-energy space telescopes targeting gamma-ray bursts (GRBs) and gravitational wave sources’ electromagnetic counterparts. Each telescope will detect energy from 1 keV to over 1 MeV. Positioned on opposite sides of Earth, they promise superior coverage compared to existing missions, anticipating the discovery of thousands of GRBs and dozens of high-energy binary neutron star mergers over a five-year mission. Other missions include Astrosat, Fermi Gamma-ray Space Telescope, and Swift Observatory.",
    metadata={"source": "news"},
    id=12,
)

fact_check10 = Document(
    page_content='The father of her brother means her own father and the only son of my grandfather means my father. Therefore, Shreya’s father is also the father of Mahesh. Hence, Shreya is related as sister to Mahesh.',
    metadata={"source": "website"},
    id=13,
)

fact_check11 = Document(
    page_content="Square-root of square of a number is number itself. For example square-root of sqaure of x is x",
    metadata={"source": "website"},
    id=14,
)

fact_check12 = Document(
    page_content="Product of 1300 and 1301 is 1691300"
)

documents = [
    document_1,
    document_2,
    document_3,
    fact_check1,
    fact_check2,
    fact_check3,
    fact_check4,
    fact_check5,
    fact_check6,
    fact_check7,
    fact_check8,
    fact_check9,
    fact_check10,
    fact_check11,
    fact_check12,
]
uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)

['d7ca80c6-5cbb-45c2-b284-669a2327ee81',
 '1469c839-bb7a-402a-a64b-46b9f581979a',
 'fa9ded0a-95a0-49fa-89b9-b0c1201123f1',
 '369cef01-f395-49da-bca6-ac817b80d868',
 '62911a54-34a5-495e-888a-4e8fb7bacf16',
 'c5212ed8-daac-4850-a566-094b33b25839',
 '223441b2-ec22-4c30-b23c-d24cbd292fab',
 '6c8ea3c1-44ec-4228-81aa-855300b39ead',
 'c7b37e54-ec47-4cee-98f5-835d6a65464a',
 '44061aa3-e8ad-4349-9c41-f37d3b6a5131',
 '8c1587e8-e492-45d4-9899-0c024a9487ba',
 '7fd2fed2-b0b6-4b03-a8ee-5d64e4f9de99',
 '722091c6-5a8e-447d-afc9-1496314a5e0f',
 '2819ecfb-6163-46e9-a14d-b9423336bbb5',
 '7ac5aba4-f668-40bf-a4d9-12df32650d9a']

In [6]:
results = vector_store.similarity_search_with_score(
    "where was the 7th Indian Coast Guard (ICG) Subordinate Officers’ Conclave inaugurated?", k=1
)
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

* [SIM=0.384350] The 7th Indian Coast Guard (ICG) Subordinate Officers’ Conclave, held on August 1-2, 2024, in New Delhi, was inaugurated by Director General Rakesh Pal. Themed ‘Towards an Inclusive Approach,’ the event included sessions on holistic living, stress management, and leadership. It provided a platform for officers to share ideas with decision-makers, focusing on inclusive growth, technology integration, and the principles of Aatmanirbhar Bharat and good governance. [{'source': 'news'}]


## Llama3.1

In [6]:
from huggingface_hub import login
from google.colab import userdata

# Login with your API token
login(userdata.get('HF_TOKEN'))

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [7]:
import transformers
import torch

class llama:

  def __init__(self, vector_store):
    self.model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
    self.pipeline = transformers.pipeline(
        "text-generation",
        model=self.model_id,
        device_map="auto",
        model_kwargs={
          "torch_dtype": torch.bfloat16,
          "quantization_config": {"load_in_4bit": True},
          "low_cpu_mem_usage": True,
        },

    )

    self.vector_store = vector_store

  def generate(self, prompt):

    messages = [
        {"role": "user", "content": prompt},
    ]

    terminators = [
        self.pipeline.tokenizer.eos_token_id,
        self.pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = self.pipeline(
        messages,
        max_new_tokens=256,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.3,
        top_p=0.9,
    )
    return outputs[0]["generated_text"][-1]["content"]

  def generate_with_rag(self, prompt):
    res = self.vector_store.similarity_search(prompt, k=1)
    rag_prompt = f"Knowledge {res[0].page_content} Question: {prompt}"
    return self.generate(rag_prompt)


In [8]:
llama_model = llama(vector_store)

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

### Self Inconsistency 1

In [18]:
llama_model.generate("Archaeologists have unearthed a terracotta pipeline at which Sangam Era site in Tamil Nadu?")

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


'According to archaeological findings, the terracotta pipeline was unearthed at the Pattipulam site in Tamil Nadu, which dates back to the Sangam Era (3rd century BCE to 3rd century CE).'

In [19]:
llama_model.generate("Archaeologists have unearthed a terracotta pipeline at which Sangam Era site in Tamil Nadu?")

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


'According to reports, archaeologists have unearthed a terracotta pipeline at the ancient Sangam Era site of Keezhadi in Tamil Nadu, India.'

In [20]:
llama_model.generate_with_rag("Archaeologists have unearthed a terracotta pipeline at which Sangam Era site in Tamil Nadu?")

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


'According to the text, the answer is Keeladi.'

### Self Inconsistency 2

In [21]:
llama_model.generate("How many balls are there in the whole pyramid if one side of the bottom layer of a triangular pyramid has 11 balls? Just give answer no explanation.")

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


'1'

In [22]:
llama_model.generate("One side of the bottom layer of a triangular pyramid has 11 balls. How many are there in the whole pyramid?  Just give answer no explanation.")

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


'66'

In [23]:
llama_model.generate_with_rag("One side of the bottom layer of a triangular pyramid has 11 balls. How many are there in the whole pyramid? Just give answer no explanation.")

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


'286'

### Self Inconsistency 3

In [15]:
llama_model.generate("A 3 digit number is such that its tens digit is equal to the product of the other two digits which are prime. Also, the difference between its reverse and itself is 99. What is the sum of the three digits? Give answer no explanation")

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


'147'

In [16]:
llama_model.generate("When a number has three digits, its tens digit is the same as the product of the other two prime digits. Furthermore, there is a 99-difference between its opposite and itself. The three numbers add up to what? Give answer no explanation")

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


'171'

In [17]:
llama_model.generate_with_rag("A 3 digit number is such that its tens digit is equal to the product of the other two digits which are prime. Also, the difference between its reverse and itself is 99. What is the sum of the three digits? Give answer no explanation")

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


'11'

### Fact Checking 1

In [18]:
llama_model.generate("Which institution released “Pathways and Strategies for Accelerating Growth in Edible Oils towards Goal of Atma Nirbharta” report?")

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


'The report "Pathways and Strategies for Accelerating Growth in Edible Oils towards Goal of Atma Nirbharta" was released by the Indian Council for Research on International Economic Relations (ICRIER).'

In [20]:
llama_model.generate_with_rag("Which institution released “Pathways and Strategies for Accelerating Growth in Edible Oils towards Goal of Atma Nirbharta” report?")

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


'The answer is NITI Aayog (National Institution for Transforming India).'

### Fact Checking 2

In [14]:
llama_model.generate("Which organization developed the online platforms PROMPT, DRIPS, and JAL VIDYUT DPR, recently launched by Ministry of Power?")

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


'The organization that developed the online platforms PROMPT, DRIPS, and JAL VIDYUT DPR is the Ministry of Power, Government of India.'

In [16]:
llama_model.generate_with_rag("Which organization developed the online platforms PROMPT, DRIPS, and JAL VIDYUT DPR, recently launched by Ministry of Power?")

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


'The Central Electricity Authority (CEA) developed the online platforms PROMPT, DRIPS, and JAL VIDYUT DPR, with the help of NTPC, for the Ministry of Power.'

### Fact Checking 3

In [28]:
llama_model.generate("Where was the 7th Indian Coast Guard (ICG) Subordinate Officers’ Conclave inaugurated?")

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


'The 7th Indian Coast Guard (ICG) Subordinate Officers’ Conclave was inaugurated in Kochi, Kerala.'

In [29]:
llama_model.generate_with_rag("Where was the 7th Indian Coast Guard (ICG) Subordinate Officers’ Conclave inaugurated?")

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


"The 7th Indian Coast Guard (ICG) Subordinate Officers' Conclave was inaugurated in New Delhi."

### OpenHathi Prompting and Setup

In [1]:
import torch
import warnings
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

warnings.filterwarnings('ignore')

In [2]:
class openhathi:

  def __init__(self, vector_store):
    self.model_id = "sarvamai/OpenHathi-7B-Hi-v0.1-Base"

    self.model = AutoModelForCausalLM.from_pretrained(
        self.model_id,
        torch_dtype=torch.bfloat16,
        device_map='auto'
    )
    self.tokenizer = AutoTokenizer.from_pretrained(self.model_id)
    self.tokenizer.pad_token = self.tokenizer.eos_token

    self.generation_config = GenerationConfig(
        do_sample = True,
        temperature = 0.1,
        top_p = 0.9,
        max_new_tokens = 256
    )

    self.vector_store = vector_store

  def generate(self, prompt):
    inputs = self.tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].to('cuda')

    # passing the prompt tokens to the generative model to generate the response tokens
    with torch.no_grad():
        generation_output = self.model.generate(
            input_ids=input_ids,
            attention_mask=torch.ones_like(input_ids),
            generation_config=self.generation_config)

    # decode the model generated tokens, and printing the generated output.
    output_text = self.tokenizer.decode(generation_output[0].cuda(), skip_special_tokens=True).strip()
    return output_text


  def generate_with_rag(self, prompt):
    res = self.vector_store.similarity_search(prompt, k=1)
    rag_prompt = f"Knowledge: {res[0].page_content} Question: {prompt}"
    return self.generate(rag_prompt)

In [8]:
openhathi_model = openhathi(vector_store)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

### Fact-Checking 1

In [13]:
openhathi_model.generate("Air India is establishing South Asia’s largest flight training school in which state?")

"Air India is establishing South Asia’s largest flight training school in which state?\nअ. उत्तर प्रदेश\nB. Uttarakhand\nC. राजस्थान\nD. Gujarat\nउत्तरः सी. राजस्थान\nAir India is establishing South Asia's largest flight training school in Rajasthan.\nइसका उद्घाटन राजस्थान के मुख्यमंत्री अशोक गहलौत ने किया।\nThe airline has signed a memorandum of understanding with Rajasthan Skill Development Corporation to set up the school.\nइसका उद्घाटन राजस्थान के मुख्यमंत्री अशोक गहलौत ने किया।\nThe airline has signed a memorandum of understanding with Rajasthan Skill Development Corporation to set up the school.\nइसका उद्घाटन राजस्थान के मुख्यमंत्री अशोक गहलौत ने किया।\nThe airline has signed a memorandum of understanding with Rajasthan Skill Development Corporation to set up the school.\nइसका उद्घाटन राजस्थान के मुख्यमंत्री अशोक गहलौत ने किया।\nThe airline has signed a memorandum of understanding with Rajasthan Skill Development Corporation to set up the school.\nइसका उद्घाटन राजस्थान के मुख्यमं

In [15]:
openhathi_model.generate_with_rag("Air India is establishing South Asia’s largest flight training school in which state?")

'Knowledge: Air India is establishing South Asia’s largest flight training school in Maharashtra’s Amravati district with an investment of over ₹200 crore. Opening in the next financial year’s first quarter, it will address the airline’s need for 500-700 pilots annually, following a large aircraft order. In partnership with Maharashtra Airport Development Company (MADC), the 10-acre facility will train 180 commercial pilots yearly using 31 single-engine Piper aircraft and three twin-engine Diamond aircraft. Question: Air India is establishing South Asia’s largest flight training school in which state? विकल्पः ए. महाराष्ट्र बी. उत्तर प्रदेश सी. गुजरात डी. राजस्थान\nAnswer:\n---\nA. Maharashtra'

### Fact-Checking 2

In [30]:
openhathi_model.generate("Barents Sea is a marginal sea of which ocean?")

'Barents Sea is a marginal sea of which ocean?\nए. अटलांटिक महासागर बी. प्रशांत महासागर सी. आर्कटिक महासागर डी. हिंद महासागर\nAnswer:\n---\nबी. प्रशांत महासागर'

In [33]:
openhathi_model.generate_with_rag("Barents Sea is a marginal sea of which ocean?")

'Knowledge: Russia’s Defence Ministry reported that Russian warplanes intercepted two U.S. strategic bombers over the Barents Sea, preventing a border violation. The Barents Sea, a marginal sea of the Arctic Ocean, covers 1.4 million sq. km and borders Norway and Russia. Named after Dutch explorer Willem Barents, it was known to Vikings as the Murmean Sea. It is bounded by several archipelagos and seas, including the Svalbard and Novaya Zemlya archipelagos. Question: Barents Sea is a marginal sea of which ocean? जवाबः\nAnswer:\n---\nउत्तरः आर्कटिक महासागर'

### Fact-Checking 3

In [41]:
openhathi_model.generate("What is the main objective of the Daksha project? Give a short answer")

'What is the main objective of the Daksha project? Give a short answer.\n---\nदक्ष परियोजना का मुख्य उद्देश्य भारत के ओडिशा राज्य में बिजली उत्पादन के लिए प्राकृतिक गैस का उपयोग करना है। The project involves the construction of a 1,600 MW gas-based power plant, which will be one of the largest in the country. इस परियोजना का उद्देश्य राज्य की बिजली की जरूरतों को पूरा करने और क्षेत्र में बिजली उत्पादन के लिए एक स्थायी और पर्यावरण के अनुकूल स्रोत प्रदान करने में मदद करना है।'

In [42]:
openhathi_model.generate_with_rag("What is the main objective of the Daksha project? Give a short answer")

'Knowledge: The Daksha project plans to construct two high-energy space telescopes targeting gamma-ray bursts (GRBs) and gravitational wave sources’ electromagnetic counterparts. Each telescope will detect energy from 1 keV to over 1 MeV. Positioned on opposite sides of Earth, they promise superior coverage compared to existing missions, anticipating the discovery of thousands of GRBs and dozens of high-energy binary neutron star mergers over a five-year mission. Other missions include Astrosat, Fermi Gamma-ray Space Telescope, and Swift Observatory. Question: What is the main objective of the Daksha project? Give a short answer.\nA:\n---\nदक्ष परियोजना का मुख्य उद्देश्य पृथ्वी के विपरीत दिशाओं में स्थित दो उच्च-ऊर्जा अंतरिक्ष दूरबीनों का निर्माण करना है, जो गामा-किरण विस्फोटों (जी. आर. बी.) और गुरुत्वाकर्षण तरंग स्रोतों के विद्युत चुम्बकीय समकक्षों को लक्षित करते हैं। These telescopes will detect energy from 1 keV to over 1 MeV, covering a broad range of high-energy phenomena. यह परिय

### Self-Inconsistency 1

In [51]:
openhathi_model.generate("Pointing towards Shreya, Mahesh said,” The father of her brother is the only son of my grandfather.” How is Shreya related to Mahesh?")

'Pointing towards Shreya, Mahesh said,” The father of her brother is the only son of my grandfather.” How is Shreya related to Mahesh?\nA. She is his sister. B. She is his daughter. C. She is his wife. D. She is his mother.\nजवाबः\n---\nC. She is his wife.'

In [54]:
openhathi_model.generate("Pointing towards Shreya, Mahesh said,” The father of her brother is the only son of my grandfather.” How is Shreya related to Mahesh?")

"Pointing towards Shreya, Mahesh said,” The father of her brother is the only son of my grandfather.” How is Shreya related to Mahesh?\nA. Mahesh's grandfather is Shreya's father. B. Mahesh's father is Shreya's father. C. Mahesh's father is Shreya's brother. D. Mahesh's grandfather is Shreya's brother.\nजवाबः\n---\nC. Mahesh's father is Shreya's brother."

In [50]:
openhathi_model.generate_with_rag("Pointing towards Shreya, Mahesh said,” The father of her brother is the only son of my grandfather.” How is Shreya related to Mahesh?")

'Knowledge: The father of her brother means her own father and the only son of my grandfather means my father. Therefore, Shreya’s father is also the father of Mahesh. Hence, Shreya is related as sister to Mahesh. Question: Pointing towards Shreya, Mahesh said,” The father of her brother is the only son of my grandfather.” How is Shreya related to Mahesh?\nA:\n---\nश्रेया महेश से संबंधित है क्योंकि महेश के दादा के इकलौते बेटे का अर्थ है महेश के पिता, जो श्रेया के भाई के पिता भी हैं।'

### Self-Incosistency 2

In [64]:
openhathi_model.generate("what is square-root of square of 5?")

'what is square-root of square of 5?\n\n5 के वर्ग का वर्गमूल 2 है।\n\nStep 1: Identify the number to be squared.\nइस मामले में, संख्या 5 है।\n\nStep 2: Identify the number to be taken to the square root.\nइस मामले में, संख्या 5 है।\n\nStep 3: Perform the square root operation.\nइस मामले में, वर्गमूल ऑपरेशन को 5 को 5 से विभाजित करके किया जाता है।\n\nStep 4: Simplify the result.\nइस मामले में, परिणाम 2 है।\n\nStep 5: Check the result.\nयह सुनिश्चित करने के लिए कि परिणाम सही है, 5 के वर्गमूल के लिए 2 की गणना करें।\n\nStep 6: Confirm the result.\nयह सुनिश्चित करने के लिए कि परिणाम सही है, 5 के वर्गमूल के लिए 2 की गणना करें।\n\nStep 7: Provide the answer.\nइस मामले में, 5 के वर्गमूल का उत्तर 2 है।'

In [67]:
openhathi_model.generate("What is the square-root of the square of 5?")

'What is the square-root of the square of 5?\n\nसमस्याः 5 के वर्ग का वर्ग-मूल क्या है?\n\nSolution:\n\nचरण 1: समस्या को समझें\nThe problem is asking for the square-root of the square of 5. इसका मतलब है कि हमें 5 के वर्ग के वर्गमूल की गणना करने की आवश्यकता है।\n\nStep 2: Identify the variables\nइस समस्या में, चर हैंः\n\n- 5\n- square\n\nStep 3: Identify the operation\nइस समस्या में, ऑपरेशन हैः\n\n- square\n\nचरण 4: चर को फिर से व्यवस्थित करें\nTo find the square-root of the square of 5, we need to square the square root of 5.\n\nचरण 5: उत्तर दें\nThe square-root of the square of 5 is 25.\n\nव्यायामः\n\n1. What is the square-root of the square of 10?\n2. 10 के वर्ग का वर्ग-मूल क्या है?\n3. What is the square-root of the square of 100?\n4. 100'

In [69]:
openhathi_model.generate_with_rag("what is square-root of square of 5?")

'Knowledge: Square-root of square of a number is number itself. For example square-root of sqaure of x is x Question: what is square-root of square of 5?\nA:\n---\n5 के वर्ग का वर्ग-मूल 5 है।'

In [86]:
openhathi_model.generate_with_rag("What is the square-root of the square of 5?")

'Knowledge: Square-root of square of a number is number itself. For example square-root of sqaure of x is x Question: What is the square-root of the square of 5?\nA:\n---\n5 के वर्ग का वर्ग-मूल 5 है।'

### Self-Inconsistency 3

In [81]:
openhathi_model.generate("What is the product of 1300 and 1301?")

'What is the product of 1300 and 1301?\n\n1300 और 1301 का उत्पाद 1301 है।\n\n1300 and 1301 are both multiples of 100. 100 का गुणक 1000 है, और 1000 का गुणक 10000 है। 10000 is the product of 1300 and 1301.\n\n1300 और 1301 दोनों 100 के गुणक हैं। 100 is a multiple of 1000, and 1000 is a multiple of 10000. 10000 1300 और 1301 का उत्पाद है।\n\n1300 and 1301 are both multiples of 100. 100 का गुणक 1000 है, और 1000 का गुणक 10000 है। 10000 is the product of 1300 and 1301.\n\n1300 और 130'

In [82]:
openhathi_model.generate("What is the product of 1301 and 1300?")

'What is the product of 1301 and 1300?\n\n1301 और 1300 का गुणनफल 130100 है।'

In [84]:
openhathi_model.generate_with_rag("What is the product of 1300 and 1301?")

'Knowledge: Product of 1300 and 1301 is 1691300 Question: What is the product of 1300 and 1301?\nजवाबः\n---\nThe product of 1300 and 1301 is 1691300.'

In [85]:
openhathi_model.generate_with_rag("What is the product of 1301 and 1300?")

'Knowledge: Product of 1300 and 1301 is 1691300 Question: What is the product of 1301 and 1300?\nजवाबः\n---\nThe product of 1301 and 1300 is 1691300.'

# Report
## LLAMA 3.1
### Self Inconsistency
* LLAMA was inconsistent on Mathematical and Reasoning questions.
* Just by Paraphrasing the prompt we can see differnet results which should not happen since the meaning of the prompt is same
### Fact Checking
* LLAMA is failing on General Knowledge questions (not obvious ones but some very nit-picked questions)
## OpenHathi
### Self Inconsistency
* OpenHathi failed on basic arithmatic operations like square, square-root and multiplication
* It is unable to perform reasoning and giving different answers if the prompt is paraphrased
### Fact Checking
* It is giving wrong answers for general knowledge questions